# 로컬에서 학습시키는 스크립트

* 데이터 manipulation 작업도 하는 곳
* 학습에 관한 여러가지 테스트도 수행할 수 있음

In [1]:
import cv2
import tensorflow as tf
import h5py
import numpy as np
import time

In [2]:
class CNN:
    
    def __init__(self, h, w, sess):
        self.size_h = h
        self.size_w = w
        self.sess = sess
        self.model = self.make_model()
        
    def make_model(self):
        
        # žðµš
        self.observation = tf.placeholder(shape=[None, self.size_h, self.size_w, 1], dtype=tf.float32) # ÀÌ¹ÌÁö µ¥ÀÌÅÍ
        self.label = tf.placeholder(shape=[None, 3], dtype=tf.int16) # ¶óº§ µ¥ÀÌÅÍ
        # L1 ImgIn shape=(?,self.size_h, self.size_w,1)
        self.w_in = tf.Variable(tf.random_normal([5,5,1,8], stddev=.01)) # conv 1 °¡ÁßÄ¡
        self.l1 = tf.nn.conv2d(self.observation, self.w_in, strides=[1,1,1,1], padding='SAME')
        #    Conv     -> (?, self.size_h, self.size_w, 8)
        self.l1 = tf.nn.relu(self.l1)
        self.l1 = tf.nn.max_pool(self.l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        #    Pool     -> (?, self.size_h/2, self.size_w/2, 8)
        
        # L2 ImgIn shape=(?,self.size_h/2, self.size_w/2,8)
        self.w2_in = tf.Variable(tf.random_normal([5,5,8,16], stddev=.01)) # conv 2 °¡ÁßÄ¡
        self.l2 = tf.nn.conv2d(self.l1, self.w2_in, strides=[1,1,1,1], padding='SAME')
        #    Conv     -> (?, self.size_h/2, self.size_w/2, 16)
        self.l2 = tf.nn.relu(self.l2)
        self.l2 = tf.nn.max_pool(self.l2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        #    Pool     -> (?, self.size_h/4, self.size_w/4, 16)
           
        '''
        # L3 ImgIn shape=(?,self.size_h/4, self.size_w/4,16)
        self.w3_in = tf.Variable(tf.random_normal([5,5,8,16], stddev=.01)) # conv 2 °¡ÁßÄ¡
        self.l3 = tf.nn.conv2d(self.l2, self.w3_in, strides=[1,1,1,1], padding='SAME')
        #    Conv     -> (?, self.size_h/4, self.size_w/4, 16)
        self.l3 = tf.nn.relu(self.l3)
        self.l3 = tf.nn.max_pool(self.l3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        #    Pool     -> (?, self.size_h/8, self.size_w/8, 16)
        
        '''
        
        self.w_out = tf.Variable(tf.random_normal(shape=[self.size_w//4*self.size_h//4*16, 3], stddev=0.01))
        self.b = tf.Variable(tf.random_normal([3]))
        self.h_flat = tf.reshape(self.l2, [-1, self.size_w//4*self.size_h//4*16])
        self.output = tf.matmul(self.h_flat, self.w_out) + self.b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.output, labels=self.label))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=.0005).minimize(self.cost)
        print('> žðµš »ýŒº ¿Ï·á')
        
        
    def train(self, batch_in, batch_label):
        # ÇÐœÀœÃÇà.
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict={self.observation:batch_in, self.label:batch_label})
        return cost
    
    def policy(self, input_data):
        # policy žž µµÃâÇÏ±â
        output = self.sess.run(self.output, feed_dict={self.observation:input_data})
        return output
    
    def test(self, in_test, label_test):
        # Å×œºÆ® µ¥ÀÌÅÍ ŒÂÀ» ¹Ìž® ÀúÀåÇØ³õ°í °ËÁõÇÏŽÂ žÞŒ­µå
        in_test_copied = copy.deepcopy(in_test)
        cnt = 0
        for step in range(len(in_test_copied)):
            key = cv2.waitKey(10) & 0xFF
            if(key == ord('q')):
                break
            out = self.sess.run(self.output, feed_dict={self.observation:in_test_copied[step:step+1]})
            idx = np.argmax(out, axis=1)
            one_hot = np.zeros_like(out)
            one_hot[0][idx[0]] = 1
            '''if(np.all(one_hot == label_test[])):
                cnt += 1'''
            cv2.putText(in_test_copied[step], 'command : {}'.format(one_hot[0]), (10, 30), cv2.FONT_HERSHEY_COMPLEX, .3, (0,0,255), 1)
            cv2.imshow('testing..', in_test_copied[step])
        cv2.destroyAllWindows()
        accuracy = cnt / len(in_test)
        print('acc :', accuracy)
        
    def test_live(self):
        # œÇœÃ°£Àž·Î ±æÀ» Àß ÀÎœÄÇÏŽÂÁö Å×œºÆ®ÇÏŽÂ žÞŒ­µå.
        out = self.sess.run(self.output, feed_dict={self.observation:d})

## 데이터 forming 

In [3]:
with h5py.File('dataset.h5', 'r') as f: # binary 파일로 학습데이터 다루기
    batch_in = np.array(f['observation'])
    batch_in = np.expand_dims(batch_in, axis=3)
    batch_in.dtype = np.uint8
    batch_label = np.array(f['label'])
    print('> 사전학습 데이터 {} frame, load & setting 완료'.format(len(batch_in)))
    print(batch_in.shape)
    #print(list(batch_in[10]), batch_label[10])

> 사전학습 데이터 1016 frame, load & setting 완료
(1016, 320, 640, 4)


## 추가 데이터 확인

In [4]:
with h5py.File('dataset_new.h5', 'r') as f:
    batch_in_new = np.array(f['observation'])
    batch_in_new = np.expand_dims(batch_in_new, axis=3)
    batch_in_new.dtype = np.uint8
    batch_label_new = np.array(f['label'])
    print('> 추가학습 데이터 {} frame, load & setting 완료'.format(len(batch_in_new)))
    print(batch_in_new.shape)
    batch_in_new = list(batch_in_new[:52]) + list(batch_in_new[59:61]) + list(batch_in_new[64:90])
    batch_label_new = list(batch_label_new[:52]) + list(batch_label_new[59:61]) + list(batch_label_new[64:90])
    print('> 편집 후 frame : ', len(batch_in_new))

> 추가학습 데이터 103 frame, load & setting 완료
(103, 320, 640, 4)
> 편집 후 frame :  80


## 데이터 보강 - 데이터 합칠 때만 사용

In [5]:
batch_in_aggre = list(batch_in) + list(batch_in_new)
batch_label_aggre = list(batch_label) + list(batch_label_new)

batch_in = np.array(batch_in_aggre)
batch_label = np.array(batch_label_aggre)

print(batch_in.shape)
print('> 최종 데이터 {} frame, 생성 완료'.format(len(batch_in)))
#print(list(batch_in[10]), batch_label[10])

(1096, 320, 640, 4)
> 최종 데이터 1096 frame, 생성 완료


## 추가된 데이터를 그대로 파일에 저장 - 필요할 때만

In [ ]:
with h5py.File('dataset.h5', 'r') as f:
    del(f['observation'])
    del(f['label'])
    f['observation'] = batch_in_aggre
    f['label'] = batch_label_aggre
    
    print('> 데이터 추가하여 {} frame, 저장 완료'.format(len(batch_in_aggre)))

## 데이터 디버깅 및 확인

In [25]:
for idx in range(len(batch_in_new)):
    #print(batch_label[idx+1016])
    cv2.putText(batch_in_new[idx], str(batch_label_new[idx]) + ' ' + str(idx), (50,50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (255,255,255), 2)
    cv2.imshow('s', batch_in_new[idx])
    if(cv2.waitKey(0) == ord('q')):
        break

cv2.destroyAllWindows()

In [18]:
with h5py.File('dataset_new.h5', 'r') as f:
    batch_in_new = np.array(f['observation'])
    batch_in_new = np.expand_dims(batch_in_new, axis=3)
    batch_in_new.dtype = np.uint8
    batch_label_new = np.array(f['label'])
    
    batch_in_new = list(batch_in_new[:52]) + list(batch_in_new[59:61]) + list(batch_in_new[64:90])
    batch_label_new = list(batch_label_new[:52]) + list(batch_label_new[59:61]) + list(batch_label_new[64:90])
    print(len(batch_in_new))

80


## 새로 학습

In [ ]:
width = 320
height = 640
total_epoch = 10
save_path = ''

with tf.Session() as sess:
    start = time.time()
    model = CNN(width, height, sess)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    print('> 데이터 학습 중..')
    for epoch in range(total_epoch):
        start_epoch = time.time()
        cost = model.train(batch_in, batch_label)
        print('{}/{} epoch, 손실크기 : {}, {} 초 소요'.format(epoch+1, total_epoch, cost, time.time() - start_epoch))
        
    save_path = 'C:\\model'
    saver.save(sess, save_path) # 모델 저장
    took_time = time.time() - start
    print('> 학습완료, {} 초 소요, 모델의 파라미터가 {} 에 저장됨.'.format(took_time, save_path))

> 모델 생성 완료
> 데이터 학습 중..
1/10 epoch, 손실크기 : 21.85662841796875, 118.28120422363281 초 소요
2/10 epoch, 손실크기 : 114.87541961669922, 99.36302947998047 초 소요
3/10 epoch, 손실크기 : 143.36880493164062, 94.52641296386719 초 소요
4/10 epoch, 손실크기 : 21.604408264160156, 103.03260660171509 초 소요
5/10 epoch, 손실크기 : 2.703319787979126, 88.62847709655762 초 소요
6/10 epoch, 손실크기 : 1.826216697692871, 91.45965623855591 초 소요
7/10 epoch, 손실크기 : 1.0754129886627197, 86.58218693733215 초 소요
8/10 epoch, 손실크기 : 0.9419257640838623, 90.66830825805664 초 소요
9/10 epoch, 손실크기 : 0.98337322473526, 83.32266998291016 초 소요
10/10 epoch, 손실크기 : 0.9921904802322388, 91.43422293663025 초 소요


## 추가 학습

In [ ]:
width = 320
height = 640
total_epoch = 10
save_path = ''

with tf.Session() as sess:
    start = time.time()
    model = CNN(width, height, sess)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, save_path)
    print('> 파라미터 load 완료')
    print('> 추가 학습 시작..')
    
    for epoch in range(total_epoch):
        start_epoch = time.time()
        cost = model.train(batch_in, batch_label)
        print('{}/{} epoch, 손실크기 : {}, {} 초 소요'.format(epoch+1, total_epoch, cost, time.time() - start_epoch))
        
    save_path = 'C:\\model'
    saver.save(sess, save_path) # 모델 저장
    took_time = time.time() - start
    print('> 학습완료, {} 초 소요, 모델의 파라미터가 {} 에 저장됨.'.format(took_time, save_path))